In [1]:
import itertools
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transformers
import matplotlib.pyplot as plt

In [2]:
class Encoder(nn.Module): 
    def __init__(self, bottleneck_size):
        super(Encoder,self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(5,3), stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=32,out_channels=64, kernel_size=(5,3), stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=64,out_channels=128, kernel_size=3,stride=2, padding=0),
            nn.ReLU()
        )

        self.dense_layers = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(128 * 30* 12 , 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512,bottleneck_size),
           )
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.flatten(x)
        x = self.dense_layers(x)
        return x


In [3]:
class Decoder(nn.Module):
    def __init__(self, bottleneck_size ):
        super(Decoder,self).__init__()
        self.conv_layers = nn.Sequential(
            nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, stride=2, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=64,out_channels=32, kernel_size=(5,3), stride=2, padding=1, output_padding=(1,0)),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=32,out_channels=1, kernel_size=(5,3),stride=2, padding=1, output_padding=1),
            nn.ReLU()
        )

        self.unflatten = nn.Unflatten(dim=1, unflattened_size=(128,30,12))

        self.dense_layers = nn.Sequential(

            nn.Linear(bottleneck_size,1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 128 * 30 * 12),
            nn.ReLU(),
            nn.Dropout(0.2),
           )

    def forward(self, x):
        x = self.dense_layers(x)
        x  = self.unflatten(x)
        x = self.conv_layers(x)
        x = x[:, :, 4:254, 2:102]

        return x


In [4]:
class AE(nn.Module):

    def __init__(self, encoder, decoder, bottleneck_size=1024):
        super(AE,self).__init__()
        self.encoder = encoder(bottleneck_size)
        self.decoder = decoder(bottleneck_size)
    def forward(self,x ):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [5]:
ae_pretrain = AE(Encoder, Decoder)

In [6]:
ae_pretrain.load_state_dict(torch.load('./logging/model.pth'))

<All keys matched successfully>

In [7]:
ae_pretrain.eval()

AE(
  (encoder): Encoder(
    (conv_layers): Sequential(
      (0): Conv2d(1, 32, kernel_size=(5, 3), stride=(2, 2), padding=(1, 1))
      (1): ReLU()
      (2): Conv2d(32, 64, kernel_size=(5, 3), stride=(2, 2), padding=(1, 1))
      (3): ReLU()
      (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
      (5): ReLU()
    )
    (dense_layers): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=46080, out_features=512, bias=True)
      (2): ReLU()
      (3): Dropout(p=0.2, inplace=False)
      (4): Linear(in_features=512, out_features=1024, bias=True)
    )
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (decoder): Decoder(
    (conv_layers): Sequential(
      (0): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(2, 2), output_padding=(1, 1))
      (1): ReLU()
      (2): ConvTranspose2d(64, 32, kernel_size=(5, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 0))
      (3): ReLU()
      (4): ConvTranspose2d(32, 1, kernel_size=(5, 3), s

In [10]:
from utils.data_loading import CombustionSystemDataset

PATH = 'combustion_img_13.mat'

test_data = CombustionSystemDataset(PATH,'test_set_x','test_set_y')
train_data = CombustionSystemDataset(PATH,'train_set_x','train_set_y')

In [11]:
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=128, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=128, shuffle=False)

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [13]:
ae_pretrain.to(device)

AE(
  (encoder): Encoder(
    (conv_layers): Sequential(
      (0): Conv2d(1, 32, kernel_size=(5, 3), stride=(2, 2), padding=(1, 1))
      (1): ReLU()
      (2): Conv2d(32, 64, kernel_size=(5, 3), stride=(2, 2), padding=(1, 1))
      (3): ReLU()
      (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
      (5): ReLU()
    )
    (dense_layers): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=46080, out_features=512, bias=True)
      (2): ReLU()
      (3): Dropout(p=0.2, inplace=False)
      (4): Linear(in_features=512, out_features=1024, bias=True)
    )
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (decoder): Decoder(
    (conv_layers): Sequential(
      (0): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(2, 2), output_padding=(1, 1))
      (1): ReLU()
      (2): ConvTranspose2d(64, 32, kernel_size=(5, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 0))
      (3): ReLU()
      (4): ConvTranspose2d(32, 1, kernel_size=(5, 3), s

In [14]:
for train_images, train_targets in train_loader:
    train_images, train_targets = train_images.to(device), train_targets.to(device)
    train_codes = ae_pretrain.encoder(train_images)

In [15]:
# Encode the data set
for test_images, test_targets in test_loader:
    test_images, test_targets = test_images.to(device), test_targets.to(device)
    test_codes = ae_pretrain.encoder(test_images)

In [18]:
print("The type of test_codes: ", type(test_codes))
print("The shape of test_codes: ", test_codes.size())
print(test_codes)

The type of test_codes:  <class 'torch.Tensor'>
The shape of test_codes:  torch.Size([80, 1024])
tensor([[ 8.3057, -4.3067, -3.1787,  ...,  0.6365, -3.8197, -0.5044],
        [ 8.5171, -2.8883, -5.1477,  ...,  0.0204, -5.6053, -0.4960],
        [ 8.4950, -1.1487, -6.6302,  ...,  0.1722, -6.3929, -0.8301],
        ...,
        [ 1.7611, -0.4993,  0.8951,  ...,  5.5987,  3.8893,  3.3396],
        [ 4.0443, -0.7045, -0.0785,  ...,  5.9880,  3.8353,  3.8628],
        [ 4.5795, -1.5750,  0.2997,  ...,  4.9544,  4.0809,  3.8103]],
       device='cuda:0', grad_fn=<AddmmBackward0>)


In [19]:
train_codes_cpu=train_codes.to('cpu')

In [20]:
train_targets_cpu=train_targets.to('cpu')

In [21]:
train_codes_cpu.detach()

tensor([[-0.1438, -7.4518,  3.2077,  ...,  7.9158,  0.4963,  0.1278],
        [ 2.5845, -7.0720,  1.5720,  ...,  7.1211,  0.8078, -0.4916],
        [ 4.8499, -7.3134,  0.9801,  ...,  5.1529,  1.4799, -2.2108],
        ...,
        [ 5.8279, -0.2781, -0.5588,  ..., -0.3481, -3.6336, -1.8335],
        [ 4.6863, -0.7095,  0.1403,  ..., -0.0320, -2.6692, -0.8317],
        [ 3.6458, -1.3339,  0.6453,  ...,  1.3926, -1.4769, -0.0316]])

In [22]:
train_codes_cpu.detach().size()

torch.Size([112, 1024])

In [23]:
imagecode = train_codes_cpu.detach().numpy()[20]

In [24]:
print(len(imagecode))

1024


In [25]:
len(train_codes_cpu)

112

In [26]:
train_codes_cpu.detach()[1]

tensor([ 2.5845, -7.0720,  1.5720,  ...,  7.1211,  0.8078, -0.4916])

In [27]:
train_targets_cpu.detach()[30]

tensor([0.])

In [39]:
def sliding_windows(data, seq_length):
    print("shape of data: ", data.size())
    print("seq_length: ", seq_length)
    
    batch_len=len(data)
    
    x = torch.empty((1,1024))
    # y = torch.empty((1,1))
    y = torch.empty(1024)
    
    for i in range(batch_len-seq_length-1):
        # _x = data.detach()[i:(i+seq_length)]
        # _y = data.detach()[i+seq_length]
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        print("Size of x: ", x.size())
        print("Size of _x: ", _x.size())
        print("Size of y: ", y.size())
        print("Size of _y: ", _y.size())

        torch.cat((x, _x),dim=0)
        torch.cat((y, _y),dim=0)

    # return torch.tensor(x),torch.tensor(y)
    return x.clone().detach(), y.clone().detach()

In [40]:
seq_length = 4
x_train, y_train = sliding_windows(train_codes_cpu, seq_length)

shape of data:  torch.Size([112, 1024])
seq_length:  4
Size of x:  torch.Size([1, 1024])
Size of _x:  torch.Size([4, 1024])
Size of y:  torch.Size([1024])
Size of _y:  torch.Size([1024])
Size of x:  torch.Size([1, 1024])
Size of _x:  torch.Size([4, 1024])
Size of y:  torch.Size([1024])
Size of _y:  torch.Size([1024])
Size of x:  torch.Size([1, 1024])
Size of _x:  torch.Size([4, 1024])
Size of y:  torch.Size([1024])
Size of _y:  torch.Size([1024])
Size of x:  torch.Size([1, 1024])
Size of _x:  torch.Size([4, 1024])
Size of y:  torch.Size([1024])
Size of _y:  torch.Size([1024])
Size of x:  torch.Size([1, 1024])
Size of _x:  torch.Size([4, 1024])
Size of y:  torch.Size([1024])
Size of _y:  torch.Size([1024])
Size of x:  torch.Size([1, 1024])
Size of _x:  torch.Size([4, 1024])
Size of y:  torch.Size([1024])
Size of _y:  torch.Size([1024])
Size of x:  torch.Size([1, 1024])
Size of _x:  torch.Size([4, 1024])
Size of y:  torch.Size([1024])
Size of _y:  torch.Size([1024])
Size of x:  torch.Siz

In [41]:
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers, device):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size)).to(device)
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size)).to(device)
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out

In [42]:
use_cuda = torch.cuda.is_available()
print(use_cuda)

True


In [43]:
num_epochs = 10
learning_rate = 0.001

input_size = 1024
hidden_size = 512
num_layers = 1

num_classes = 2

# locate our GPU device and initialize it for training
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

lstm = LSTM(num_classes, input_size, hidden_size, num_layers, device)
lstm.to(device)
print("Number of parameters: ", sum(p.numel() for p in lstm.parameters()))

criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    outputs = lstm(trainX.to(device))
    optimizer.zero_grad()
    
    # obtain the loss function
    loss = criterion(outputs, trainY.to(device))
    
    loss.backward()
    
    optimizer.step()
    if epoch % 100 == 0:
      print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

Number of parameters:  3150850


NameError: name 'trainX' is not defined